In [1]:
#!/usr/bin/env python
#-*- coding:utf-8 -*-
############################
#File Name: userBased.py
#Author: yanbin
#Mail: yanbin918@gmail.com
#Created Time: 2017-08-17 10:12:02
############################
import math,operator
import sys
sys.path.append('../')
from ml_latest_small.data import *


class UserCFModel(object):
    def __init__(self,train,W,K=5,N=10):
        self.train = train
        self.W = W
        self.K = K
        self.N = N


    def recommendation(self,user_id,config=None):
        W = self.W
        K = self.K
        N = self.N
        if config is not None:
            try:
                K = config['K']
                N = config['N']
            except:
                print "config = {'K':nearest nums,'N':topN nums}"
                return
        train = self.train
        rank = dict()
        interacted_items = train[user_id]
        for v,wuv in sorted(W[user_id].items(),key=operator.itemgetter(1),
                            reverse=True)[0:K]:
            for i , rvi in train[v].items():
                if i in interacted_items:
                    continue
                if i not in rank:
                    rank[i] = 0
    #filter items user interacted
                rank[i] += wuv*rvi
        topN = sorted(rank.items(),key=operator.itemgetter(1),reverse=True)[0:N]
        return dict(topN)



class UserCF(object):
    def userSimilarity(self,train):
        W=dict()
        for u in train.keys():
            W[u]=dict()
            for v in train.keys():
                if u==v:
                    continue
                W[u][v]=len(set(train[u].keys())
		&set(train[v].keys()))/math.sqrt(len(train[u])*len(train[v])*1.0)
        return W

    def userSimilarity2(self,train):
        #build inverse table for item_users
        item_users = dict()
        for u,items in train.items():
            for i in items.keys():
                if i not in item_users:
                    item_users[i] = set()
                item_users[i].add(u)
        #calculate co-rated items between users
        C = dict()
        N = dict()
        for i, users in item_users.items():
            for u in users:
                if u not in C:
                    C[u]=dict()
                if u not in N:
                    N[u]=0
                N[u] +=1
                for v in users:
                    if u==v:
                        continue
                    if v not in C[u]:
                        C[u][v]=0
                    C[u][v]+=1
        #calculate finial similarity matrix W
        W = dict()
        for u,related_users in C.items():
            W[u]=dict()
            for v,cuv in related_users.items():
                W[u][v] = cuv/math.sqrt(N[u] * N[v])
        return W


    def userSimilarity3(self,train):
        #build inverse table for item_users
        item_users = dict()
        for u,items in train.items():
            for i in items.keys():
                if i not in item_users:
                    item_users[i] = set()
                item_users[i].add(u)

        #calculate co-rated items between users
        C = dict()
        N = dict()
        for i, users in item_users.items():
            for u in users:
                if u not in C:
                    C[u]=dict()
                if u not in N:
                    N[u]=0
                N[u] +=1
                for v in users:
                    if u==v:
                        continue
                    if v not in C[u]:
                        C[u][v]=0
                    C[u][v]+=1/math.log(1+len(users))
        #calculate finial similarity matrix W
        W = dict()
        for u,related_users in C.items():
            W[u]=dict()
            for v,cuv in related_users.items():
                W[u][v] = cuv/math.sqrt(N[u] * N[v])
        return W

    def train(self,train,Normalization=False):
        W = self.userSimilarity(train)
        if Normalization :
            W = self.Normalization(W)
        return UserCFModel(train,W)

    def Normalization(self,W):
        for i,items in W.items():
            m = max(items.values())
            for j,wij in items.items():
                W[i][j]=wij/m
        return W

if __name__ == '__main__':
    userCF = UserCF()
    inputPath = "../ml_latest_small/ratings.csv"
    train_dataset,test_dataset = read_data_sets(inputPath,with_split=True)
    train = train_dataset.user_item_dict
    UserCF_model=userCF.train(train)
    print UserCF_model.recommendation(user_id='405')

{'592': 0.9735029549398699, '3623': 0.9735029549398699, '1722': 0.9735029549398699, '5445': 1.2043742039028251, '1375': 0.9735029549398699, '1610': 0.9735029549398699, '736': 1.2043742039028251, '2194': 0.9735029549398699, '500': 1.2043742039028251, '480': 1.2043742039028251}


In [4]:
W = UserSimilarity2(train)
GetRecommendation('2',10)
# Recommend('1',data,W)

# Popularity(data,data,N)
Ks=[5,10,20,40,80,160]
N=10
for k in Ks:
    K=k
    recall = Recall(train,test,N)
    precision = Precision(train,test,N)
    coverage = Coverage(train,test,N)
    popularity = Popularity(train,test,N)
    print ("k:%3d,准确率：%2.2f%%，召回率:%2.2f%%，覆盖率:%2.2f%%，流行度:%2.2f" 
        %(K,precision*100,recall*100,coverage*100,popularity))

k: 5,准确率：21.25%，召回率:11.98%，覆盖率:42.55%，流行度:5.12
k:10,准确率：25.26%，召回率:14.24%，覆盖率:32.47%，流行度:5.28
k:20,准确率：28.03%，召回率:15.80%，覆盖率:25.17%，流行度:5.40
k:40,准确率：29.10%，召回率:16.40%，覆盖率:19.13%，流行度:5.49
k:80,准确率：29.64%，召回率:16.71%，覆盖率:14.06%，流行度:5.56
k:160,准确率：28.26%，召回率:15.93%，覆盖率:11.10%，流行度:5.61


In [8]:
class topnModel(object):
    def __init__(self,items_times=dict(),N=10)
        self.items_times = items_times
        self.N = N
        
    def train(train)
        for user,items in train.items():
            for item,times in items.items():
                if item not in items_times:
                    items_times[item]=0
                items_times[item]+=1

    def recommendation(self,user_id,config=None):
        N = self.N 
        if config is not None:
            N = config['N']
        rank = sorted(items_times.items(),key=operator.itemgetter(1),reverse=True)[0:N]
        return dict(rank)
    
Ks=[5,10,20,40,80,160]
N=10
topNs=topN(train,N)
for k in Ks:
    K=k
    recall = Recall(train,test,N)
    precision = Precision(train,test,N)
    coverage = Coverage(train,test,N)
    popularity = Popularity(train,test,N)
    print ("准确率：%2.2f%%，召回率:%2.2f%%，覆盖率:%2.2f%%，流行度:%2.2f" 
        %(precision*100,recall*100,coverage*100,popularity))



准确率：4.78%，召回率:4.99%，覆盖率:0.60%，流行度:6.09
准确率：4.78%，召回率:4.99%，覆盖率:0.60%，流行度:6.09
准确率：4.78%，召回率:4.99%，覆盖率:0.60%，流行度:6.09
准确率：4.78%，召回率:4.99%，覆盖率:0.60%，流行度:6.09
准确率：4.78%，召回率:4.99%，覆盖率:0.60%，流行度:6.09
准确率：4.78%，召回率:4.99%，覆盖率:0.60%，流行度:6.09


In [7]:
class RandomModel(object):
    def __init__(self,train,N=10):
        self.tarin = train
        self.N = N
        
    def recommendation(self,user_id,config=None):
        N = self.N
        if config is not None:
            try:
                K = config['K']
                N = config['N']
            except:
                print "config = {'K':nearest nums,'N':topN nums}"
                return
        train = set(self.train.values()
        result=[]
        for i in range(N):
            rd=random.randint(0,len(train)-1)
            result.append([train[rd][-1]])
        return result
    
class RandomRM(object):
    def __init__(self,train,N=10):
        self.tarin = train
        self.N = N
    
    def train(self)
    
    
def GetRecommendation(user,N):
    return Random(train2,N)
data = loadData(inputPath)
train2,test = SplitData(data,10,0,40)
train = toDict(train2)
test = toDict(test)
Ks=[5,10,20,40,80,160]
N=10
for k in Ks:
    K=k
    recall = Recall(train,test,N)
    precision = Precision(train,test,N)
    coverage = Coverage(train,test,N)
    popularity = Popularity(train,test,N)
    print ("k:%3d,准确率：%2.2f%%，召回率:%2.2f%%，覆盖率:%2.2f%%，流行度:%2.2f" 
        %(K,precision*100,recall*100,coverage*100,popularity))

k:  5,准确率：1.53%，召回率:1.92%，覆盖率:73.64%，流行度:4.71
k: 10,准确率：1.53%，召回率:1.78%，覆盖率:73.22%，流行度:4.70
k: 20,准确率：1.59%，召回率:1.67%，覆盖率:72.56%，流行度:4.70
k: 40,准确率：1.59%，召回率:1.66%，覆盖率:71.96%，流行度:4.72
k: 80,准确率：1.43%，召回率:1.39%，覆盖率:72.74%，流行度:4.71
k:160,准确率：1.55%，召回率:1.75%，覆盖率:72.98%，流行度:4.70


$$W_{ij}=\frac{\lvert N(i)\bigcap N(j)\rvert}{\lvert N(i) \rvert}$$ 
$$W_{uv}=\frac{\lvert N(u)\bigcap N(v)\rvert}{\sqrt{\lvert N(u) \rvert \lvert N(v) \rvert}}$$
$$w_{ij}=\frac{\sum_{u\in N(i) \bigcap N(j)}\frac{1}{log(1+\lvert N(u)\rvert)}}{\sqrt{\lvert N(i) \rvert \lvert N(j) \rvert}}$$
$$w_{uv}=\frac{\sum_{i\in N(u) \bigcap N(v)}\frac{1}{1}}{\sqrt{\lvert N(u) \rvert \lvert N(v) \rvert}}$$

$$w^{'}_{ij}=\frac{w_{ij}}{\max\limits_{j}w_{ij}}$$

$$Recall = \frac{\sum_{u\in U}\lvert R(u) \bigcap T(u)\rvert}{\sum_{u \in U}\lvert T(u)\rvert}$$


$$Precision = \frac{\sum_{u\in U}\lvert R(u) \bigcap T(u)\rvert}{\sum_{u \in U}\lvert R(u)\rvert}$$


$$ Coverage=\frac{\vert\bigcup_{u \in U}R(u)\vert}{\vert I \vert}$$


$$ Popularity=\frac{1}{\sum\limits_{u\in U}\vert R(u)\vert}\sum\limits_{u\in U}\sum\limits_{i\in R(u)}{log(1+N(i))}$$